# **Classification de ressentis avec CamemBERT**

L'objectif est de créer un modèle qui prend en entrée des commentaires (en Français) et attribue à chacun un ressenti positif ou négatif.  
Le modèle global est composé de deux parties :  
* [CamemBERT](https://camembert-model.fr/) va encoder le commentaire et en extraire des informations qui seront passées ensuite au réseau de neurones.  
* Le modèle suivant est un réseau de neurones qui sera créé avec l'API [Keras](https://www.tensorflow.org/guide/keras?hl=fr) de [Tensorflow](https://www.tensorflow.org/?hl=fr).  

<img src="https://raw.githubusercontent.com/AlexandreBourrieau/ML-F1/master/Carnets%20Jupyter/Images/SchemaCamembert1.png" />  
  
  Les données qui s'échangent entre les deux modèles sont des vecteurs de dimension 768. On peut voir ces vecteurs comme l'équivalent de l'application d'un algorithme de prolongation lexicale sur les mots qui composent le commentaire.

In [111]:
!pip install transformers --quiet

In [113]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, Input, Dropout, Lambda
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

from transformers import CamembertConfig
from transformers import TFCamembertModel
from transformers import AutoTokenizer

import matplotlib.pyplot as plt

# Importation des données

Téléchargement des ressentis Allociné

In [ ]:
# Téléchargement des données depuis le repot github "https://github.com/AlexandreBourrieau/ML-F1/raw/master/Carnets%20Jupyter/Donn%C3%A9es/data.tar.bz2"

!wget -q "https://github.com/AlexandreBourrieau/ML-F1/raw/master/Carnets%20Jupyter/Donn%C3%A9es/data.tar.bz2"
!tar -xjvf data.tar.bz2

In [ ]:
df = pd.read_json("/content/data/test.jsonl", lines=True)
df.head(5)

Affiche quelques informations :

In [116]:
def LongueurMax(df):
  Lmax = 0
  for com in df['review']:
    Longueur = len(com)
    if Lmax < Longueur:
      Lmax = Longueur
  return Lmax

In [ ]:
print(df[0:10])
print("Total des données : ", str(len(df)))
print("Nombre d'avis positifs et négatifs : ",df['polarity'].value_counts())
print("Longueur maximale d'un commentaire : ",LongueurMax(df))

On ne va garder que les commentaires dont la longueur est inférieure à une certaine valeur :

In [118]:
L_MAX = 300

df2 = df[df['review'].str.len()<L_MAX]

# **Préparation des données**


In [119]:
def LongueurMax2(df):
  Lmax = 0
  for com in df:
    Longueur = len(com)
    if Lmax < Longueur:
      Lmax = Longueur
  return Lmax

In [120]:
# Chargement des commentaires et des ressentis
commentaires = df2['review'].astype(str).tolist()    # Récupère tous les commentaires dans une liste python
ressentis = df2['polarity'].tolist()                   # Récupère tous les ressentis dans une liste python
labels = np.asarray(ressentis)               # Créé un tableau de type numpy avec les ressentis

x_entrainement, x_test, y_entrainement, y_test = train_test_split(commentaires[0:MAX], labels[0:MAX], test_size=0.25)

In [ ]:
Longueur_max_entrainement = LongueurMax2(x_entrainement)
Longueur_max_tests = LongueurMax2(x_test)
print("Longueur maximale des commentaires : ", max([Longueur_max_entrainement, Longueur_max_tests]))

In [ ]:
print ("Nombre de commentaires pour l'entrainement : ", len(x_entrainement))
print ("Nombre de commentaires pour les tests : ", len(x_test))

# **Tokénisation**


In [123]:
LONGUEUR_MAX_COMMENTAIRE = max([Longueur_max_entrainement, Longueur_max_tests])


# Instanciation du tokeniseur
tokenizer = AutoTokenizer.from_pretrained('jplu/tf-camembert-base')

# Préparation des données d'entrainement
output_tokenizer_entrainement = tokenizer(x_entrainement,max_length=LONGUEUR_MAX_COMMENTAIRE, padding='max_length', truncation=False, return_tensors='tf',add_special_tokens=True)

# Préparation des données de tests
output_tokenizer_tests = tokenizer(x_test,max_length=LONGUEUR_MAX_COMMENTAIRE, padding='max_length', truncation=False, return_tensors='tf',add_special_tokens=True)

Regardons un peu comment sont formatées les données en sortie du tokéniseur :

In [ ]:
output_tokenizer_entrainement

Regardons comment le premier commentaire a été encodé :

In [ ]:
print("Commentaire original :", x_entrainement[0])
print("input_ids: ", output_tokenizer_entrainement['input_ids'][0])
print("attention_mask: ", output_tokenizer_entrainement['attention_mask'][0])

# **Définition et utilisation du modèle camemBERT avec Keras**

In [ ]:
# Instanciation du modèle camemBERT
transformer_model = TFCamembertModel.from_pretrained('jplu/tf-camembert-base')

# Défintion du format des entrées du modèle
entrees_ids = tf.keras.layers.Input(shape=(LONGUEUR_MAX_COMMENTAIRE,), name='input_token', dtype='int32')
entrees_masks = tf.keras.layers.Input(shape=(LONGUEUR_MAX_COMMENTAIRE,), name='masked_token', dtype='int32') 

# Création de la sortie du modèle
sortie_camemBERT = transformer_model([entrees_ids,entrees_masks])

# Instanciation du modèle avec Keras
model_camemBERT = tf.keras.Model(inputs=[entrees_ids, entrees_masks], outputs = sortie_camemBERT,trainable=False)
model_camemBERT.summary()

Si on regarde le format de la sortie du modèle camemBERT, on voit qu'elle est composée de deux sorties :
* Une sortie avec un format (None,MAX_SEQUENCE_LENGTH,768)
* Une sortie avec un format (None,768)  
  
On trouve la signification de ces sorties [sur le site de hugginface](https://huggingface.co/transformers/main_classes/output.html#tfbasemodeloutput). Ainsi, la première sortie est de type `last_hidden_state` et la deuxième de type `pooler_output`.

In [ ]:
sortie_camemBERT

 Celle qui nous interesse ici est la sortie `last_hidden_state` : C'est elle qui contient le résultat de l'encodage des mots des commentaires

In [ ]:
sortie_camemBERT[0]

La fonction `predict()` permet d'exécuter le modèle sur les séquences d'entrées

In [ ]:
sortie_vecteurs_camemBERT = model_camemBERT.predict(
    [output_tokenizer_entrainement['input_ids'][0:2],
     output_tokenizer_entrainement['attention_mask'][0:2]]
     ,verbose=1)

Regardons à quoi ressemble la sortie de camemBERT :

In [ ]:
sortie_vecteurs_camemBERT

# **Ajout du réseau de neurones simple en sortie du modèle camemBERT**

**Extraction des vecteurs [CLS]**

Parmi les MAX_SEQUENCE_LENGTH vecteurs en sortie, il ne nous faut que le premier (celui qui correspond au mot clé [CLS]). On doit donc récupérer, pour chaque commentaire, le premier vecteur de dimension 768 parmi les MAX_SEQUENCE_LENGTH en sortie :  

In [ ]:
sortie_camemBERT[0]

**Construction du modèle global**

Les vecteurs de dimension 768 correspondants aux sorties [CLS] de chaque commentaire sont envoyés dans un réseau de neurones à 2 neurones avec une fonction d'activation Softmax :
<img src="https://raw.githubusercontent.com/AlexandreBourrieau/ML-F1/master/Carnets%20Jupyter/Images/SchemaCamembert2.png"/>

In [ ]:
# Instanciation du modèle camemBERT
transformer_model = TFCamembertModel.from_pretrained('jplu/tf-camembert-base')

# Défintion du format des entrées du modèle
entrees_ids = tf.keras.layers.Input(shape=(LONGUEUR_MAX_COMMENTAIRE,), name='input_token', dtype='int32')
entrees_masks = tf.keras.layers.Input(shape=(LONGUEUR_MAX_COMMENTAIRE,), name='masked_token', dtype='int32') 

# Création de la sortie du modèle
sortie_camemBERT = transformer_model([entrees_ids,entrees_masks])

# Instanciation du modèle avec Keras
model_camemBERT = tf.keras.Model(inputs=[entrees_ids, entrees_masks], outputs = sortie_camemBERT,trainable=False)

# Défintion du format des entrées du modèle
entrees_ids = tf.keras.layers.Input(shape=(LONGUEUR_MAX_COMMENTAIRE,), name='input_token', dtype='int32')
entrees_masks = tf.keras.layers.Input(shape=(LONGUEUR_MAX_COMMENTAIRE,), name='masked_token', dtype='int32') 

# Création de la sortie du modèle
sortie_camemBERT = transformer_model([entrees_ids,entrees_masks])[0]


l1 = Lambda(lambda seq: seq[:, 0, :])(sortie_camemBERT)        # On ne récupère que les vecteurs [CLS]
output = Dense(2, activation='softmax')(l1)

model = tf.keras.Model(inputs=[entrees_ids, entrees_masks], outputs = output)
model.layers[2].trainable = False         # Désactive d'entrainement de camemBERT

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

On lance maintenant l'entrainement du modèle :

In [ ]:
history = model.fit([output_tokenizer_entrainement['input_ids'],output_tokenizer_entrainement['attention_mask']],y_entrainement,
                    epochs=5, verbose=1, batch_size = 3,
                    validation_data=([output_tokenizer_tests['input_ids'],output_tokenizer_tests['attention_mask']],y_test))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Précision du modèle')
plt.ylabel('Précision')
plt.xlabel('Itération')
plt.legend(['Entrainement', 'Test'], loc='upper left')
plt.show()

# **Fine Tuning**

Afin d'obtenir une meilleur précision, on va également entrainer camemBERT : 

In [ ]:
model.layers[2].trainable = True
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(1e-5), metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit([output_tokenizer_entrainement['input_ids'],output_tokenizer_entrainement['attention_mask']],y_entrainement,
                    epochs=5, verbose=1, batch_size = 3,
                    validation_data=([output_tokenizer_tests['input_ids'],output_tokenizer_tests['attention_mask']],y_test))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Précision du modèle')
plt.ylabel('Précision')
plt.xlabel('Itération')
plt.legend(['Entrainement', 'Test'], loc='upper left')
plt.show()